In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# Yvou can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [2]:
!pip install transformers peft bitsandbytes accelerate datasets torch sentencepiece


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 69.7/69.7 MB 18.9 MB/s eta 0:00:0000:0100:01m


In [1]:
import os
token=os.getenv("HF_TOKEN")


In [ ]:
from huggingface_hub import login
token=os.getenv("HF_TOKEN")
login(token)


In [7]:
from datasets import load_dataset

ds = load_dataset("thu-coai/esconv")

README.md:   0%|          | 0.00/510 [00:00<?, ?B/s]

train.txt:   0%|          | 0.00/4.04M [00:00<?, ?B/s]

valid.txt:   0%|          | 0.00/865k [00:00<?, ?B/s]

test.txt:   0%|          | 0.00/881k [00:00<?, ?B/s]

Generating train split:   0%|          | 0/910 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/195 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/195 [00:00<?, ? examples/s]

In [8]:
train_data=ds["train"]["text"]
test_data = (ds["test"]["text"])
validation_data = (ds["validation"]["text"])

In [9]:
import json
import pandas as pd

# Suppose your raw data is stored in the variable `data`
# where each element is a JSON string (one conversation per line).
# For example:
# data = [
#    '{"experience_type": "Current Experience", "emotion_type": "anxiety", ... }',
#    '{"experience_type": "...", ... }',
#    ...
# ]
def create_dataset(data, save_path):
    training_samples = []  # To store the transformed examples
    
    for line in data:
        # Parse the JSON for one conversation
        conv = json.loads(line)
        dialog = conv["dialog"]
    
        # First, merge consecutive utterances by the same speaker.
        merged_blocks = []  # Each block is a dict: {speaker, text, [strategies]}
        for utt in dialog:
            speaker = utt["speaker"].strip()
            text = utt["text"].strip()
            strategy = utt.get("strategy")  # might be None for user turns
    
            if merged_blocks and merged_blocks[-1]["speaker"] == speaker:
                # Same speaker as previous block → merge texts.
                merged_blocks[-1]["text"] += " " + text
                # For system turns, if a strategy is present, add it to the list.
                if speaker == "sys" and strategy:
                    merged_blocks[-1]["strategies"].append(strategy)
            else:
                # Start a new block.
                if speaker == "sys":
                    # For sys blocks, store the strategy in a list.
                    merged_blocks.append({
                        "speaker": speaker,
                        "text": text,
                        "strategies": [strategy] if strategy else []
                    })
                else:
                    merged_blocks.append({
                        "speaker": speaker,
                        "text": text
                    })
    
        # Next, generate training samples.
        # The idea is: for each system (sys) block (which carries a strategy),
        # create a sample whose input text is the conversation history _before_ that sys block.
        for i, block in enumerate(merged_blocks):
            if block["speaker"] == "sys" and block.get("strategies"):
                # Build the conversation context from all previous blocks.
                context_parts = []
                for b in merged_blocks[:i]:
                    if b["speaker"] == "usr":
                        context_parts.append(f"usr: {b['text']}")
                    elif b["speaker"] == "sys":
                        # Join the strategies with a comma if there are multiple.
                        strat = ", ".join(b.get("strategies", []))
                        context_parts.append(f"sys({strat}): {b['text']}")
                context = " ".join(context_parts).strip()
    
                # The label is the strategy (or list of strategies) from the current sys block.
                training_samples.append({
                    "text": context,
                    "label": block["strategies"]
                })
    
    # Optionally, save the transformed data as a CSV (each row: text, label)
    df = pd.DataFrame(training_samples)
    df.to_csv(save_path, index=False)
    
    # For inspection, print the first few samples:
    for sample in training_samples[:2]:
        print("Text:", sample["text"])
        print("Label:", sample["label"])
        print("------")


In [10]:
import json
import pandas as pd

def create_dataset(data, save_path):
    training_samples = []  # To store the transformed examples
    
    for line in data:
        # Parse the JSON for one conversation
        conv = json.loads(line)
        dialog = conv["dialog"]
    
        # First, merge consecutive utterances by the same speaker.
        merged_blocks = []  # Each block is a dict: {speaker, text, [strategies]}
        for utt in dialog:
            speaker = utt["speaker"].strip()
            text = utt["text"].strip()
            strategy = utt.get("strategy")  # might be None for user turns
    
            if merged_blocks and merged_blocks[-1]["speaker"] == speaker:
                # Same speaker as previous block → merge texts.
                merged_blocks[-1]["text"] += " " + text
                # For system turns, if a strategy is present, add it to the list.
                if speaker == "sys" and strategy:
                    merged_blocks[-1]["strategies"].append(strategy)
            else:
                # Start a new block.
                if speaker == "sys":
                    # For sys blocks, store the strategy in a list.
                    merged_blocks.append({
                        "speaker": speaker,
                        "text": text,
                        "strategies": [strategy] if strategy else []
                    })
                else:
                    merged_blocks.append({
                        "speaker": speaker,
                        "text": text
                    })
    
        # Next, generate training samples.
        for i, block in enumerate(merged_blocks):
            if block["speaker"] == "sys" and block.get("strategies"):
                # Build the conversation context from the previous utterances (max 6).
                context_parts = []
                start_idx = max(0, i - 4)  # Keep only the last 6 utterances
                
                for b in merged_blocks[start_idx:i]:
                    if b["speaker"] == "usr":
                        context_parts.append(f"usr: {b['text']}")
                    elif b["speaker"] == "sys":
                        strat = ", ".join(b.get("strategies", []))
                        context_parts.append(f"sys({strat}): {b['text']}")
                
                context = " ".join(context_parts).strip()
                if context is not None:
                    training_samples.append({
                        "text": context,
                        "label": block["strategies"]
                    })
    
    # Save the transformed data as a CSV
    df = pd.DataFrame(training_samples)
    df.to_csv(save_path, index=False)
    
    # Print first few samples for inspection
    for sample in training_samples[:2]:
        print("Text:", sample["text"])
        print("Label:", sample["label"])
        print("------")


In [11]:
create_dataset(train_data, "train.csv")
create_dataset(test_data, "test.csv")
create_dataset(validation_data, "val.csv")

Text: usr: Hello good afternoon.
Label: ['Question']
------
Text: usr: Hello good afternoon. sys(Question): Hi, good afternoon. usr: I'm feeling anxious that I am going to lose my job.
Label: ['Reflection of feelings']
------
Text: 
Label: ['Others']
------
Text: sys(Others): Hello. How are you today? usr: hi i am okay, a little bit sad though
Label: ['Question']
------
Text: 
Label: ['Question']
------
Text: sys(Question): Hi, what I can help you with today? usr: Hi I had problems in the past and somehow right now
Label: ['Restatement or Paraphrasing']
------


In [12]:
# from datasets import load_dataset

# # Example: Load a CSV dataset with "text" and "label" columns
# dataset = load_dataset("csv", data_files={"train": "train.csv", "test": "test.csv", "val": "val.csv"})
# dataset = dataset.filter(lambda example: example["text"] is not None)
# print(dataset["train"][10])  # Check a sample


In [13]:
# import pandas as pd

# # Load the dataset
# df = pd.read_csv("val.csv")

# # Compute the max number of words in the "text" column
# max_words = df["text"].apply(lambda x: len(str(x).split())).max()

# print("Maximum number of words in 'text':", max_words)


In [14]:
# from transformers import AutoModelForCausalLM, AutoTokenizer
# import torch

# model_name = "meta-llama/Meta-Llama-3-8B"  # Or any other available LLaMA 3 model
# # model_name = "mistralai/Mistral-7B-Instruct-v0.3"
# tokenizer = AutoTokenizer.from_pretrained(model_name)
# model = AutoModelForCausalLM.from_pretrained(
#     model_name,
#     device_map="auto",
#     torch_dtype=torch.float16,
#     use_auth_token=True
# )
# tokenizer.pad_token = tokenizer.eos_token

In [15]:
# from peft import LoraConfig, get_peft_model

# lora_config = LoraConfig(
#     r=8,                    # Rank of LoRA matrices
#     lora_alpha=32,          # Scaling factor
#     target_modules=["q_proj", "v_proj"],  # Apply LoRA to attention layers
#     lora_dropout=0.1,
#     bias="none",
#     task_type="CAUSAL_LM",  # Fine-tuning for text generation/classification
# )

# model = get_peft_model(model, lora_config)
# model.print_trainable_parameters()


In [16]:
# dataset

In [17]:
# def tokenize_function(example):
#     # print (example)
#     return tokenizer(
#         example["text"],
#         padding="max_length",
#         truncation=True,
#         max_length=1024
#     )

# tokenized_datasets = dataset.map(tokenize_function, batched=False)


In [18]:
# from transformers import TrainingArguments, Trainer

# training_args = TrainingArguments(
#     output_dir="./llama3_lora_text_classification",
#     evaluation_strategy="epoch",
#     save_strategy="epoch",
#     per_device_train_batch_size=4,
#     per_device_eval_batch_size=4,
#     num_train_epochs=3,
#     logging_steps=1,  # Log more frequently
#     learning_rate=2e-4,
#     fp16=True,  # Use mixed precision training
#     save_total_limit=2,
#     push_to_hub=False,
#     report_to="none",  # Disable WandB if not needed
#     logging_dir="./logs",  # Specify log directory
#     disable_tqdm=False,  # Ensure tqdm progress bars are visible
# )

# trainer = Trainer(
#     model=model,
#     args=training_args,
#     train_dataset=tokenized_datasets["train"],
#     eval_dataset=tokenized_datasets["test"],
# )




In [19]:
# trainer.train()  # Start training with visible logs

In [20]:
# model.save_pretrained("mistral_strategy_classification")
# tokenizer.save_pretrained("mistral_strategy_classification")

# # Test
# text = "sys(Question): Hello! How are you tonight? usr: hanging in there, kinda freaking out a little sys(Affirmation and Reassurance, Question): I'm sorry to hear you are freaking out. Perhaps a friendly listening ear can encourage you? Why are you freaking out? usr: I think that'll help, yeah sys(Restatement or Paraphrasing): Sometimes just a friend to hear you out and encourage you can really make a difference. Life has its ups and down. usr: I decided to leave my husband, he keeps snapping at me and won't listen to me or let me do anything"
# inputs = tokenizer(text, return_tensors="pt").to("cuda")
# outputs = model.generate(**inputs)
# print(outputs)


In [21]:
# import ast
# from datasets import load_dataset
# from transformers import AutoTokenizer, AutoModelForSequenceClassification, TrainingArguments, Trainer, DataCollatorWithPadding
# import torch

# # ============================
# # 1. Define the Label Set
# # ============================

# # List all possible strategies (adjust if needed)
# strategy_labels = [
#     "Question",
#     "Restatement or Paraphrasing",
#     "Reflection of feelings",
#     "Self-disclosure",
#     "Affirmation and Reassurance",
#     "Providing Suggestions",
#     "Information",
#     "Others"
# ]
# num_labels = len(strategy_labels)
# # Create mapping dictionaries to convert between labels and indices.
# label2id = {label: i for i, label in enumerate(strategy_labels)}
# id2label = {i: label for i, label in enumerate(strategy_labels)}

# # ============================
# # 2. Load Your CSV Datasets
# # ============================

# # Assumes you have train.csv, val.csv, and test.csv in your working directory.
# data_files = {
#     "train": "train.csv",
#     "validation": "val.csv",
#     "test": "test.csv"
# }
# raw_datasets = load_dataset("csv", data_files=data_files)

# # ============================
# # 3. Preprocessing Function
# # ============================

# def preprocess_function(examples):
#     # Tokenize the input text using the LLaMA tokenizer.
#     # Here we use a max_length of 512; adjust as needed.
#     encoding = tokenizer(examples["text"], truncation=True, padding="max_length", max_length=1024)
    
#     # Convert the label string into a multi-hot vector.
#     multi_hot_labels = []
#     for label_str in examples["label"]:
#         # Convert string to list. For example, the string "['Question']" becomes a list.
#         try:
#             labels = ast.literal_eval(label_str)
#         except Exception:
#             # If conversion fails, treat it as a single label (or empty).
#             labels = [label_str] if label_str else ["Others"]
        
#         # Initialize a vector of zeros for each possible strategy.
#         multi_hot = [0] * num_labels
#         for lab in labels:
#             if lab in label2id:
#                 multi_hot[label2id[lab]] = 1
#         multi_hot_labels.append(multi_hot)
    
#     # Add the processed labels into the encoding.
#     encoding["labels"] = multi_hot_labels
#     return encoding
# # 

In [22]:
# raw_datasets["train"][7]

In [23]:

# # ============================
# # 4. Load the Tokenizer & Model
# # ============================

# # Replace with your LLaMA 7B model identifier.
# model_name = "MaziyarPanahi/Mistral-7B-Instruct-v0.3-GGUF"

# # Load the tokenizer.
# tokenizer = AutoTokenizer.from_pretrained(model_name, use_auth_token=True)

# # Load the model with a classification head.
# # Setting problem_type to "multi_label_classification" configures the loss as BCEWithLogitsLoss.
# model = AutoModelForSequenceClassification.from_pretrained(
#     model_name,
#     num_labels=num_labels,
#     problem_type="multi_label_classification",
#     use_auth_token=True
# )



In [24]:
# tokenizer.pad_token = tokenizer.eos_token

In [25]:
# # ============================
# # 5. Tokenize the Datasets
# # ============================

# # Map the preprocessing function over the entire dataset.
# # remove_columns removes the original columns (e.g., text and label strings) after processing.
# raw_datasets = raw_datasets.filter(lambda example: example["text"] is not None)
# tokenized_datasets = raw_datasets.map(
#     preprocess_function,
#     batched=False,
#     remove_columns=raw_datasets["train"].column_names
# )

# # Create a data collator to handle dynamic padding.
# data_collator = DataCollatorWithPadding(tokenizer)



In [26]:
# del raw_datasets
# del ds
# del dataset

In [27]:
# tokenized_datasets["train"][1]

In [28]:
# # ============================
# # 6. Set Up TrainingArguments & Trainer
# # ============================

# training_args = TrainingArguments(
#     output_dir="./llama_multi_label",
#     evaluation_strategy="epoch",
#     save_strategy="epoch",
#     learning_rate=2e-5,
#     per_device_train_batch_size=4,
#     per_device_eval_batch_size=4,
#     num_train_epochs=3,
#     weight_decay=0.01,
#     # fp16=True,  # Use mixed precision (if supported)
#     save_total_limit=2,
#     load_best_model_at_end=True,
#     report_to="none",  # Disable WandB if not needed
#     logging_dir="./logs",  # Specify log directory
#     disable_tqdm=False,  # Ensure tqdm progress bars are visible
#     logging_steps=1,  # Log more frequently
# )

# trainer = Trainer(
#     model=model,
#     args=training_args,
#     train_dataset=tokenized_datasets["train"],
#     eval_dataset=tokenized_datasets["validation"],
#     tokenizer=tokenizer,
#     data_collator=data_collator,
# )




In [29]:
# # ============================
# # 7. Fine-Tuning the Model
# # ============================

# trainer.train()

# # Save the final model and tokenizer.
# model.save_pretrained("llama_multi_label_final")
# tokenizer.save_pretrained("llama_multi_label_final")

In [30]:
import ast
import torch
import numpy as np
from datasets import load_dataset
from transformers import (
    AutoTokenizer,
    AutoModelForSequenceClassification,
    AutoModelForSeq2SeqLM,
    TrainingArguments,
    Trainer,
    DataCollatorWithPadding,
)
from sklearn.metrics import f1_score, precision_score, recall_score

# ================================
# 1. Define the Label Set & Mapping
# ================================
strategy_labels = [
    "Question",
    "Restatement or Paraphrasing",
    "Reflection of feelings",
    "Self-disclosure",
    "Affirmation and Reassurance",
    "Providing Suggestions",
    "Information",
    "Others",
]
num_labels = len(strategy_labels)
label2id = {label: i for i, label in enumerate(strategy_labels)}
id2label = {i: label for i, label in enumerate(strategy_labels)}

# ================================
# 2. Load the CSV Datasets
# ================================
data_files = {
    "train": "train.csv",
    "validation": "val.csv",
    "test": "test.csv"
}
raw_datasets = load_dataset("csv", data_files=data_files)
raw_datasets
# ================================
# 3. Preprocessing for Encoder Models
# ================================
import ast
import re

def preprocess_function(examples, tokenizer, max_length=512):
    # Process and trim conversation texts to only include the last 6 utterances
    new_texts = []
    for text in examples["text"]:
        text = str(text) # Remove if error
        # Split the conversation into utterances using a regex that looks ahead for "usr:" or "sys("
        turns = re.split(r'(?=usr:|sys\()', text)
        # Remove any empty strings and strip leading/trailing whitespace
        turns = [turn.strip() for turn in turns if turn.strip()]
        # Keep only the last 6 utterances if there are more than 6
        if len(turns) > 6:
            turns = turns[-6:]
        # Reconstruct the text from the last utterances
        new_texts.append(" ".join(turns))
    
    # Tokenize the modified texts
    encoding = tokenizer(
        new_texts,
        truncation=True,
        padding="max_length",
        max_length=max_length
    )
    
    # Convert label strings (e.g. "['Information', 'Reflection of feelings']") into multi-hot vectors.
    multi_hot_labels = []
    for label_str in examples["label"]:
        try:
            labels = ast.literal_eval(label_str)
        except Exception:
            labels = [label_str] if label_str else ["Others"]
        multi_hot = [0.0] * num_labels
        for lab in labels:
            if lab in label2id:
                multi_hot[label2id[lab]] = 1
        multi_hot_labels.append(multi_hot)
    
    encoding["labels"] = multi_hot_labels
    return encoding


The cache for model files in Transformers v4.22.0 has been updated. Migrating your old cache. This is a one-time only operation. You can interrupt this and resume the migration later on by calling `transformers.utils.move_cache()`.


0it [00:00, ?it/s]

Generating train split: 0 examples [00:00, ? examples/s]

Generating validation split: 0 examples [00:00, ? examples/s]

Generating test split: 0 examples [00:00, ? examples/s]

In [31]:


# ================================
# 4. Compute Metrics Function
# ================================
def compute_metrics(p):
    logits, labels = p
    # Apply sigmoid and threshold at 0.5 to get binary predictions
    probs = torch.sigmoid(torch.tensor(logits))
    preds = (probs > 0.5).int().numpy()
    f1 = f1_score(labels, preds, average="micro", zero_division=0)
    precision = precision_score(labels, preds, average="micro", zero_division=0)
    recall = recall_score(labels, preds, average="micro", zero_division=0)
    return {"f1": f1, "precision": precision, "recall": recall}


# ================================
# 5. Fine-Tuning Encoder-Based Models
# ================================
# Models are ordered by expected accuracy (best to lowest):
# ALBERT > DistilBERT > MiniLM > TinyBERT
encoder_models = [
    # ("microsoft/deberta-v3-large", "DeBERTa V3 Large"),   # ~1.5B parameters
    # ("microsoft/deberta-v3-xlarge", "DeBERTa V3 XLarge"),
    ("albert-base-v2", "ALBERT"),
    ("distilbert-base-uncased", "DistilBERT"),
    ("nreimers/minilm-l6-h384-uncased", "MiniLM"),
    ("huawei-noah/TinyBERT_General_4L_312D", "TinyBERT"),
    # ("microsoft/deberta-v3-base", "DeBERTa V3 Base"),  # Stronger than BERT-based models
    # ("xlm-roberta-base", "XLM-RoBERTa Base"),  # Multilingual alternative to RoBERTa
    # ("tiiuae/falcon-7b", "Falcon 7B")  # Open-source model optimized for efficiency
    # ("NousResearch/Nous-Hermes-2-Mistral-7B", "Nous Hermes 7B")
]

In [32]:
raw_datasets = raw_datasets.filter(lambda example: example["text"] is not None)

Filter:   0%|          | 0/10679 [00:00<?, ? examples/s]

Filter:   0%|          | 0/2257 [00:00<?, ? examples/s]

Filter:   0%|          | 0/2389 [00:00<?, ? examples/s]

In [33]:
# # Training loop over your encoder models.
# for checkpoint, model_name in encoder_models:
#     print(f"\n===== Fine-tuning {model_name} ({checkpoint}) =====")
#     # Load tokenizer and model with a multi-label classification head.
#     tokenizer = AutoTokenizer.from_pretrained(checkpoint)
#     model = AutoModelForSequenceClassification.from_pretrained(
#         checkpoint,
#         num_labels=num_labels,
#         problem_type="multi_label_classification",  # Uses BCEWithLogitsLoss
#     ).to("cuda")
    
#     # Preprocess the datasets for this tokenizer.
#     processed_datasets = raw_datasets.map(
#         lambda examples: preprocess_function(examples, tokenizer, max_length=512),
#         batched=True,
#         remove_columns=raw_datasets["train"].column_names
#     )
#     data_collator = DataCollatorWithPadding(tokenizer)
    
#     # Define training arguments.
#     training_args = TrainingArguments(
#         output_dir=f"./results/{model_name}",
#         num_train_epochs=20,
#         per_device_train_batch_size=16,
#         per_device_eval_batch_size=16,
#         evaluation_strategy="epoch",
#         save_strategy="epoch",
#         load_best_model_at_end=True,
#         metric_for_best_model="f1",
#         logging_steps=50,
#         logging_dir=f"./logs/{model_name}",
#         report_to="none",
#         fp16=True,  # Enables mixed precision training (saves memory)
#     )
    
#     trainer = Trainer(
#         model=model,
#         args=training_args,
#         train_dataset=processed_datasets["train"],
#         eval_dataset=processed_datasets["validation"],
#         tokenizer=tokenizer,
#         data_collator=data_collator,
#         compute_metrics=compute_metrics,
#     )
    
#     # Fine-tune and evaluate
#     trainer.train()
#     results = trainer.evaluate(processed_datasets["test"])
#     print(f"Evaluation results for {model_name}: {results}\n")
    
#     # Clean up GPU memory
#     del model, tokenizer, trainer, processed_datasets, data_collator, training_args
#     gc.collect()
#     torch.cuda.empty_cache()

In [34]:

# # ================================
# # 6. Fine-Tuning T5 (Flan-T5) for Text-to-Text Classification
# # ================================
# def preprocess_function_t5(examples, tokenizer, max_input_length=512, max_target_length=32):
#     # Add a classification prompt.
#     inputs = ["classify: " + text for text in examples["text"]]
#     model_inputs = tokenizer(
#         inputs,
#         max_length=max_input_length,
#         truncation=True,
#         padding="max_length"
#     )
#     # Convert label lists to a comma-separated string.
#     targets = []
#     for label_str in examples["label"]:
#         try:
#             labels = ast.literal_eval(label_str)
#         except Exception:
#             labels = [label_str] if label_str else ["Others"]
#         targets.append(", ".join(labels))
#     # Tokenize the target labels.
#     with tokenizer.as_target_tokenizer():
#         labels_enc = tokenizer(
#             targets,
#             max_length=max_target_length,
#             truncation=True,
#             padding="max_length"
#         )
#     model_inputs["labels"] = labels_enc["input_ids"]
#     return model_inputs



# print("\n===== Fine-tuning Flan-T5 (text-to-text formulation) =====")
# t5_checkpoint = "google/flan-t5-small"
# tokenizer_t5 = AutoTokenizer.from_pretrained(t5_checkpoint)
# model_t5 = AutoModelForSeq2SeqLM.from_pretrained(t5_checkpoint).to("cuda")

# processed_datasets_t5 = raw_datasets.map(
#     lambda examples: preprocess_function_t5(examples, tokenizer_t5, max_input_length=512, max_target_length=32),
#     batched=False
# )

# training_args_t5 = TrainingArguments(
#     output_dir="./results/Flan-T5",
#     num_train_epochs=3,
#     per_device_train_batch_size=16,
#     per_device_eval_batch_size=16,
#     evaluation_strategy="epoch",
#     save_strategy="epoch",
#     load_best_model_at_end=True,
#     logging_steps=50,
#     logging_dir="./logs/Flan-T5",
#     report_to="none",
# )

# trainer_t5 = Trainer(
#     model=model_t5,
#     args=training_args_t5,
#     train_dataset=processed_datasets_t5["train"],
#     eval_dataset=processed_datasets_t5["validation"],
#     tokenizer=tokenizer_t5,
# )

# trainer_t5.train()
# results_t5 = trainer_t5.evaluate(processed_datasets_t5["test"])
# print(f"Evaluation results for Flan-T5: {results_t5}")

# # Clean up T5-related GPU memory
# del model_t5, tokenizer_t5, trainer_t5, processed_datasets_t5, training_args_t5
# gc.collect()
# torch.cuda.empty_cache()

In [35]:
!pip install flash_attn

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.0/6.0 MB 43.2 MB/s eta 0:00:0000:0100:01
  Preparing metadata (setup.py) ... done
  Created wheel for flash_attn: filename=flash_attn-2.7.4.post1-cp310-cp310-linux_x86_64.whl size=187797312 sha256=b267f80a08e516292cdd748056a2178a45b8abedf7fca123292eb17c21c8c87c
  Stored in directory: /root/.cache/pip/wheels/59/ce/d5/08ea07bfc16ba218dc65a3a7ef9b6a270530bcbd2cea2ee1ca
Successfully built flash_attn


In [36]:
!pip show bitsandbytes
# !pip install -U bitsandbytes

Name: bitsandbytes
Version: 0.45.2
Summary: k-bit optimizers and matrix multiplication routines.
Home-page: https://github.com/bitsandbytes-foundation/bitsandbytes
Author: 
Author-email: Tim Dettmers <dettmers@cs.washington.edu>
License: MIT License
        
        Copyright (c) Facebook, Inc. and its affiliates.
        
        Permission is hereby granted, free of charge, to any person obtaining a copy
        of this software and associated documentation files (the "Software"), to deal
        in the Software without restriction, including without limitation the rights
        to use, copy, modify, merge, publish, distribute, sublicense, and/or sell
        copies of the Software, and to permit persons to whom the Software is
        furnished to do so, subject to the following conditions:
        
        The above copyright notice and this permission notice shall be included in all
        copies or substantial portions of the Software.
        
        THE SOFTWARE IS PROVIDED 

In [37]:
# import csv
# import ast
# import fasttext

# def convert_csv_to_fasttext_format(input_csv, output_txt):
#     """Converts CSV with columns 'text' and 'label' to fastText format."""
#     with open(input_csv, 'r', encoding='utf-8') as fin, open(output_txt, 'w', encoding='utf-8') as fout:
#         reader = csv.DictReader(fin)
#         for row in reader:
#             text = row['text'].strip().replace('\n', ' ')
#             label_str = row['label']
#             try:
#                 labels = ast.literal_eval(label_str)
#             except Exception:
#                 labels = [label_str]
#             # Replace spaces in labels with underscores & prefix with __label__
#             labels = ["__label__" + label.strip().replace(" ", "_") for label in labels]
#             fout.write(" ".join(labels) + " " + text + "\n")

# # Convert your CSVs (adjust paths as needed)
# convert_csv_to_fasttext_format("train.csv", "train_ft.txt")
# convert_csv_to_fasttext_format("val.csv", "val_ft.txt")
# convert_csv_to_fasttext_format("test.csv", "test_ft.txt")

# # Train a fastText supervised classifier (using one-vs-all loss for multi-label classification).
# model_ft = fasttext.train_supervised(
#     input="train_ft.txt",
#     epoch=1,
#     lr=0.1,
#     wordNgrams=2,
#     loss="ova"  # One-vs-all loss for multi-label classification.
# )

# # Evaluate on test data.
# result = model_ft.test("test_ft.txt")
# print("FastText results (N, precision, recall):", result)


In [38]:
raw_datasets["train"][10]

{'text': 'sys(Question): How are you doing today? usr: hello I am ok how are you?',
 'label': "['Question']"}

In [39]:
# import torch
# from transformers import AutoTokenizer, AutoModelForSequenceClassification, TrainingArguments, Trainer, DataCollatorWithPadding
# from datasets import load_dataset
# import ast

# # Define your label set.
# strategy_labels = [
#     "Question",
#     "Restatement or Paraphrasing",
#     "Reflection of feelings",
#     "Self-disclosure",
#     "Affirmation and Reassurance",
#     "Providing Suggestions",
#     "Information",
#     "Others"
# ]
# num_labels = len(strategy_labels)
# label2id = {label: i for i, label in enumerate(strategy_labels)}
# id2label = {i: label for i, label in enumerate(strategy_labels)}


# # Load your dataset (assuming CSV files for train, validation, and test).
# raw_datasets = load_dataset("csv", data_files={"train": "train.csv", "validation": "val.csv", "test": "test.csv"})

# # Load tokenizer and preprocess data.
# tokenizer = AutoTokenizer.from_pretrained("bert-base-uncased")
# processed_datasets = raw_datasets.map(
#     lambda examples: preprocess_function(examples, tokenizer, max_length=512),
#     batched=True,
#     remove_columns=raw_datasets["train"].column_names
# )
# data_collator = DataCollatorWithPadding(tokenizer)

# # Load the model with a multi-label classification head.
# model = AutoModelForSequenceClassification.from_pretrained(
#     "bert-base-uncased",
#     num_labels=num_labels,
#     problem_type="multi_label_classification"  # Uses BCEWithLogitsLoss internally.
# )

# # Define training arguments.
# training_args = TrainingArguments(
#     output_dir="./results/bert_classifier",
#     num_train_epochs=20,
#     per_device_train_batch_size=16,
#     per_device_eval_batch_size=16,
#     evaluation_strategy="epoch",
#     save_strategy="epoch",
#     load_best_model_at_end=True,
#     metric_for_best_model="f1",
#     logging_steps=50,
#     logging_dir="./logs/bert_classifier",
#     report_to="none",
#     fp16=True,  # Enable mixed precision training.
# )

# # Initialize the Trainer.
# trainer = Trainer(
#     model=model,
#     args=training_args,
#     train_dataset=processed_datasets["train"],
#     eval_dataset=processed_datasets["validation"],
#     tokenizer=tokenizer,
#     data_collator=data_collator,
#     compute_metrics=compute_metrics,
# )

# # Train and evaluate.
# trainer.train()
# results = trainer.evaluate(processed_datasets["test"])
# print("BERT classifier results:", results)


In [40]:
# import torch
# import gc
# from transformers import (
#     AutoTokenizer,
#     AutoModelForSequenceClassification,
#     TrainingArguments,
#     Trainer,
#     DataCollatorWithPadding,
#     BitsAndBytesConfig  # Import BitsAndBytesConfig for quantization
# )
# from peft import LoraConfig, get_peft_model, prepare_model_for_kbit_training

# # Example large model checkpoint using QLoRA (adjust as needed)
# # checkpoint = "openlm-research/open_llama_3b_v2" 4 hours per epoch # or "mistralai/Mistral-7B", etc.
# configs=[("openlm-research/open_llama_3b_v2", ["q_proj", "v_proj"]),
#         ("stanford-crfm/BioMedLM", ["c_attn", "c_proj", "c_fc", "c_ffn"])]
# # checkpoint="stanford-crfm/BioMedLM" # 4 hrs ish
# for checkpoint, target in configs:     
#     print(f"\n===== Fine-tuning with QLoRA on {checkpoint} =====")
    
#     # Set up BitsAndBytesConfig for 4-bit quantization.
#     quantization_config = BitsAndBytesConfig(
#         load_in_4bit=True,
#         bnb_4bit_compute_dtype=torch.float16,  # or torch.bfloat16 if supported
#         bnb_4bit_use_double_quant=True,
#         bnb_4bit_quant_type="nf4"  # "nf4" is common; adjust if needed
#     )
    
#     print("Quant setup...")
    
#     # Load the tokenizer
#     tokenizer = AutoTokenizer.from_pretrained(checkpoint)
#     if tokenizer.pad_token is None:
#         tokenizer.pad_token = tokenizer.eos_token
#     # Ensure the model's config uses the same padding token
#     # Load the model using quantization_config instead of load_in_4bit.
#     model = AutoModelForSequenceClassification.from_pretrained(
#         checkpoint,
#         num_labels=num_labels,
#         problem_type="multi_label_classification",
#         quantization_config=quantization_config,
#         device_map="auto",
#         # attn_implementation="flash_attention_2"
#     )
#     # model.to("cuda")
#     model.config.pad_token_id = tokenizer.pad_token_id
    
#     print("Tokenizer and Model loaded...")
    
#     # Prepare the model for k-bit training (freezes most parameters and adds adapters).
#     model.gradient_checkpointing_enable()
#     model = prepare_model_for_kbit_training(model)
    
#     # Set up a LoRA configuration. Adjust target_modules based on your model architecture.
#     lora_config = LoraConfig(
#         r=8,                # LoRA rank
#         lora_alpha=32,      # Scaling factor
#         # target_modules=["q_proj", "v_proj"],  # Typical targets; adjust if needed.
#         # target_modules=["query_key_value"],
#         # target_modules=["c_attn", "c_proj", "c_fc", "c_ffn"],  # Adjusted targets for GPT-2 architecture
#         target_modules=target,
#         lora_dropout=0.1,
#         bias="none",
#     )
#     model = get_peft_model(model, lora_config)
    
#     # LORA Setup
    
#     # Preprocess your datasets using your preprocess_function.
#     processed_datasets = raw_datasets.map(
#         lambda examples: preprocess_function(examples, tokenizer, max_length=512),
#         batched=True,
#         remove_columns=raw_datasets["train"].column_names
#     )
#     data_collator = DataCollatorWithPadding(tokenizer)
    
#     # Define training arguments.
#     training_args = TrainingArguments(
#         output_dir=f"./results/QLoRA-{checkpoint.split('/')[-1]}",
#         num_train_epochs=2,
#         per_device_train_batch_size=16,  # Use a lower batch size for large models
#         per_device_eval_batch_size=16,
#         evaluation_strategy="epoch",
#         save_strategy="epoch",
#         load_best_model_at_end=True,
#         metric_for_best_model="f1",
#         logging_steps=20,
#         logging_dir=f"./logs/QLoRA-{checkpoint.split('/')[-1]}",
#         report_to="none",
#         fp16=True,  # Enable mixed precision training
#     )
    
#     trainer = Trainer(
#         model=model,
#         args=training_args,
#         train_dataset=processed_datasets["train"],
#         eval_dataset=processed_datasets["validation"],
#         tokenizer=tokenizer,
#         data_collator=data_collator,
#         compute_metrics=compute_metrics,  # Your custom metric function
#     )
    
#     print("Trainer setup... Starting Training...")
    
#     # Fine-tune and evaluate.
#     trainer.train()
#     results = trainer.evaluate(processed_datasets["test"])
#     print(f"Evaluation results for QLoRA on {checkpoint}: {results}")
    
#     # Clean up GPU memory after training.
#     del model, tokenizer, trainer, processed_datasets, data_collator, training_args
#     gc.collect()
#     torch.cuda.empty_cache()


In [41]:
# -----------------------------
# Create a System Prompt with Chain-of-Thought Instructions
# -----------------------------
# Provided examples with chain-of-thought (you can modify these as needed).
example_prompts = """
Example 1:
Input: usr: I'm feeling anxious about finding work and impatient with the application process at many places I've applied.
sys(Restatement or Paraphrasing): So from what I am understanding, you have been applying for jobs and you are having anxiety about finding one and waiting through the application process.
usr: Yes, that's right. I keep trying and feel like I'm not getting anywhere.
sys(Reflection of feelings): Just from the few messages we have shared I can tell how anxious this is making you.
usr: Yes. I think that I might have to try to find a different type of work than I'm used to and I'm not sure I'll be good at trying something new.
sys(Self-disclosure): I just recently went through a layoff at my company, and I had the same situation trying to find another job. It is very stressful.
usr: I wish that I could get more of a reason than just a denial of my application. I'm not sure what I'm doing wrong and most companies won't give any feedback.
sys(Providing Suggestions): What have you done to reach out for feedback from the companies? Often it helps to send emails or make phone calls to have them give you some feedback.
usr: I haven't tried reaching out. Most of the places I've been applying are huge corporations with no one to call at HR. Maybe I should try to apply somewhere locally and see if I have better luck. At least then I could talk to a person.
sys(Others, Providing Suggestions): I think that would be a great start! It also might help to get business cards from the people you interview with in order to be able to reach out to them in the future.
usr: That's a great idea. I've been considering going back to school to see if I can maybe get a higher degree. I've got a Bachelor's degree now and it seems like everything takes a Master's degree these days.
sys(Question): What type of work are you interested in?
usr: What I'm most interested in is importing and exporting of exotic fruit but the recent restrictions on travel have really hurt the business. I've been thinking maybe I'll go back to school for something more practical like plumbing.
sys(Information): I think that would be a great idea! It has been shown that people with higher level degrees make more money and have more opportunity than those without.
usr: Ugh, getting accepted for a master's program sounds stressful. I'm thinking I can't go wrong by becoming a plumber or an electrician. Everyone always needs that, even with the pandemic.

Reasoning: I started the conversation started with acknowledging and validating the user’s anxiety (Reflection of feelings).
Then, I shared a personal anecdote (Self-disclosure) to build rapport and relatability.
When the user expressed frustration about lack of feedback, I suggested an actionable step (Providing Suggestions).
As the user explored options, additional suggestions (Providing Suggestions, Others) helped guide them.
The user mentioned education, so I asked a question to understand their interests (Question).
Finally, an informative response (Information) helped affirm their thoughts. I should provide more suggestions next(Providing Suggestions).

Final Answer: Providing Suggestions

Example 2:
Input: usr: hi I need help. I'm under academic stress.
sys(Question): What are you stressed about?
usr: I am failing one of my classes. I'm worried about my scholarship.
sys(Affirmation and Reassurance): Oh, Covid is really having a negative effect on a lot of students right now. Has school-from-home had a negative effect on you?
usr: Yeah, I feel like I can't focus because I'm back home with my family and I just feel like there are so many distractions.
sys(Restatement or Paraphrasing): It's interesting that you find being home more distracting than being away in a dorm.
usr: Yeah, that's really the reason I wanted to go away from college. I don't really have my own space when I'm here. People just come in and out of my room as they please despite how busy I am.
sys(Reflection of feelings, Self-disclosure, Question): Yeah, sometimes families can struggle with the idea of boundaries. I also went away to school for similar reasons, but I actually found college life way more distracting, to be honest. Have you told your family about the issues you're having?
usr: I honestly am worried that they will be disappointed. Maybe that's crazy. I just remember how proud they were when I got this scholarship. I don't want them to blame me.

Reasoning: I started with a (Question) because I needed to understand the user’s specific stressor.
Once I saw that their academic struggles were tied to their home environment, I reflected on their feelings and used self-disclosure to build rapport.
I also asked a follow-up question to help them explore a possible solution—communicating with their family.  
I think I should ask them more (Question) and let them explore their feelings (Reflection of Feelings) so I can give better Suggestions later.

Final Answer: Reflection of feelings, Question

Example 3:
Input: sys(Question): Hello! Hope you are doing well. How may I assist you?
usr: My recent ex-girlfriend gave her daughters drugs while on a video chat with me. While being very dishonest in our relationship, I am devastated about the truth of all of it now that it's over. I really loved her and her kids; we had some great times. I turned her in for giving her kids drugs as that's not okay. Today I'm super sad. I don't have any motivation. I don't really want to be around, and I'm trapped in my thoughts with everything. I had to move to get away from her stalking me. Now I'm in a new place trying to find a job and get set up here. It's difficult to start over. It's a pretty intense situation, I admit.
sys(Restatement or Paraphrasing): Your ex-girlfriend gave drugs to her own kids. Did I get that right?
usr: That is what she did. Among many other things.
sys(Affirmation and Reassurance): That is really intense and unacceptable. I feel sorry for the poor little girl and you. It looks like there is something wrong. I can understand what you are going through.
usr: It's a difficult time. I want to do nothing. It feels very bleak right now worrying about finances and getting set up in a new state.
sys(Reflection of feelings): It must be a hard time for you. You should not worry about her. You should think about your job and the poor girl who is trapped with her.
usr: Her daughters now both live with their dads, so that's taken care of; the kids are safer and better off. I, however, am devastated that the whole relationship wound up a lie and the whole thing has left me in a bad place mentally.
sys(Self-disclosure): I felt the same way when I broke up with my friend over some petty things she did. The good part is, the girls are safe now. Give some time to yourself—time will heal everything.
usr: That's a hard light to see at the end of the tunnel when things are bleak. I'm not sure what else to do. My mind runs away a bit right now; it's just all stressful.

Reasoning: The user is in a very intense situation, so I used (Restatement or Paraphrasing) to better understand their situation.
I try to assure them that it'll be okay and that they have done the right thing(Affirmation and Reassurance).
i think I need to make them less tense so I will use (Affirmation and Reassurance) to further help.

Final Answer: Affirmation and Reassurance

Example 4:
usr: I'm feeling anxious that I am going to lose my job.
sys(Reflection of feelings): Losing a job is always anxiety-provoking.
usr: I hope I don't.
sys(Question): Why do you think you will lose your job?
usr: I am on short term disability and I am not ready to go back to work yet, and I do not have any job protection.
sys(Restatement or Paraphrasing): Oh, so your job is not protected and your short term disability will end soon? Is that correct?
usr: It's not ending yet, but no, my job is not protected. I live in the United States, but I have not been at my job long enough to earn protection for medical leave. You have to have been here for a year, and I started November 2020. I'm afraid that I will lose my job since I'm still on disability for the foreseeable future.
sys(Providing Suggestions): I see. Have you spoken to HR?
usr: I have, but they are telling me that it is up to my department manager who isn't actually getting back to me about it yet.
sys(Restatement or Paraphrasing): Your department manager is not answering you?
usr: No, I have sent them a few emails about it. It makes me nervous. I do not have a phone number to call and my psychiatrist really does not think I am ready to go back to the stress of my job. I wish I could just call him, but I do not have a phone number for him. Just his email.
sys(Providing Suggestions): Have you tried mentioning that to HR?
usr: HR is the one who gave me his email. I don't think he has a direct work line, and they're not allowed to give out personal information.

Reasoning: I reflected on the user’s feelings to show empathy(Reflection of feelings), then asked a question(Question) to understand their specific concern about their job.
Once I learned about their job protection issue, I restated it to ensure I understood correctly(Restatement or Paraphrasing).
Since they seemed uncertain about what steps to take, I provided a (Suggestion) to help them explore their options.
Next step I should take is to help user explore their feelings (Reflection of feelings) and ask for more information so I can help better(Information).

Final Answer: Information, Reflection of feelings
"""

In [42]:
# import ast
# import random
# import torch
# from transformers import AutoTokenizer, AutoModelForSeq2SeqLM, 
# from sklearn.metrics import f1_score, precision_score, recall_score

# # -----------------------------
# # Configuration and Model Loading
# # -----------------------------
# # Use a larger seq2seq model.
# model_name = "NousResearch/Nous-Hermes-2-Mistral-7B-DPO"

# # Define the list of possible strategies.
# strategy_labels = [
#     "Question",
#     "Restatement or Paraphrasing",
#     "Reflection of feelings",
#     "Self-disclosure",
#     "Affirmation and Reassurance",
#     "Providing Suggestions",
#     "Information",
#     "Others"
# ]
# # Mapping for converting label to index.
# label2id = {label: i for i, label in enumerate(strategy_labels)}

# # Load tokenizer and model.
# tokenizer = AutoTokenizer.from_pretrained(model_name)
# model = AutoModelForSeq2SeqLM.from_pretrained(model_name)
# device = "cuda" if torch.cuda.is_available() else "cpu"
# model.to(device)

# # Create the system prompt by combining a task description with the examples.
# def create_system_prompt_custom():
#     task_description = (
#         "Your task is to analyze a conversation and predict the therapy strategy or strategies used. "
#         "For each conversation, provide a detailed chain-of-thought reasoning that explains how previous parts "
#         "of the conversation lead to subsequent strategies. Then, at the end, output the final predicted strategies "
#         "as a comma-separated list. The possible strategies are: Question, Restatement or Paraphrasing, Reflection of feelings, "
#         "Self-disclosure, Affirmation and Reassurance, Providing Suggestions, Information, and Others."
#     )
#     system_prompt = f"{task_description}\n\nExamples:\n{example_prompts}\nBased on the above, predict the strategies for the following conversation."
#     return system_prompt

# system_prompt = create_system_prompt_custom()
# print("=== System Prompt ===")
# print(system_prompt)
# print("=" * 50)

# # -----------------------------
# # Helper function to convert a list of labels into a multi-hot vector.
# # -----------------------------
# def convert_labels_to_vector(label_list):
#     vector = [0] * len(strategy_labels)
#     for label in label_list:
#         label = label.strip()
#         if label in label2id:
#             vector[label2id[label]] = 1
#     return vector

# # -----------------------------
# # Generate Prediction with Chain-of-Thought Reasoning
# # -----------------------------
# def generate_prediction(input_text, system_prompt, tokenizer, model, max_length=256):
#     full_prompt = f"{system_prompt}\n\nInput: {input_text}\n\nChain-of-Thought and Final Answer:"
#     inputs = tokenizer(full_prompt, return_tensors="pt", truncation=True, max_length=512)
#     inputs = {k: v.to(model.device) for k, v in inputs.items()}
    
#     output_ids = model.generate(
#         **inputs,
#         max_length=max_length,
#         num_beams=5,
#         early_stopping=True
#     )
#     prediction = tokenizer.decode(output_ids[0], skip_special_tokens=True)
#     return prediction.strip()

# # -----------------------------
# # Evaluate on Test Data and Compute Metrics
# # -----------------------------
# all_true_vectors = []
# all_pred_vectors = []

# print("\n=== Testing on Test Data ===\n")
# for idx, test_example in enumerate(raw_datasets["test"]):
#     test_input = test_example["text"]
#     expected_label_str = test_example["label"]
    
#     try:
#         true_labels = ast.literal_eval(expected_label_str)
#     except Exception:
#         true_labels = [expected_label_str]
#     true_labels = [label.strip() for label in true_labels]
    
#     generated_output = generate_prediction(test_input, system_prompt, tokenizer, model)
    
#     # Attempt to extract the final answer by searching for a marker.
#     if "Final Answer:" in generated_output:
#         final_part = generated_output.split("Final Answer:")[-1]
#         pred_labels = [lab.strip() for lab in final_part.split(",") if lab.strip()]
#     else:
#         pred_labels = [lab.strip() for lab in generated_output.split(",") if lab.strip()]
    
#     true_vector = convert_labels_to_vector(true_labels)
#     pred_vector = convert_labels_to_vector(pred_labels)
    
#     all_true_vectors.append(true_vector)
#     all_pred_vectors.append(pred_vector)
    
#     print(f"Example {idx+1}:")
#     print("Input:")
#     print(test_input)
#     print("\nChain-of-Thought and Prediction:")
#     print(generated_output)
#     print(f"\nExpected Strategies: {true_labels}")
#     print("-" * 50)

# # Compute overall metrics.
# f1 = f1_score(all_true_vectors, all_pred_vectors, average="micro", zero_division=0)
# precision = precision_score(all_true_vectors, all_pred_vectors, average="micro", zero_division=0)
# recall = recall_score(all_true_vectors, all_pred_vectors, average="micro", zero_division=0)

# print("\n=== Overall Metrics ===")
# print(f"F1 Score: {f1:.4f}")
# print(f"Precision: {precision:.4f}")
# print(f"Recall: {recall:.4f}")


In [43]:
import ast
import random
import torch
from transformers import AutoTokenizer, AutoModelForCausalLM
from sklearn.metrics import f1_score, precision_score, recall_score

# -----------------------------
# Configuration and Model Loading
# -----------------------------
# Use Mistral 7B, which is a causal LM.
model_name = "NousResearch/Nous-Hermes-2-Mistral-7B-DPO"

strategy_labels = [
    "Question",
    "Restatement or Paraphrasing",
    "Reflection of feelings",
    "Self-disclosure",
    "Affirmation and Reassurance",
    "Providing Suggestions",
    "Information",
    "Others"
]
label2id = {label: i for i, label in enumerate(strategy_labels)}

# Load tokenizer and causal model.
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForCausalLM.from_pretrained(
    model_name,
    torch_dtype=torch.float16,
    device_map="auto"
)
# device = "cuda" if torch.cuda.is_available() else "cpu"
# model.to(device)

def create_system_prompt_custom():
    task_description = (
        "Your task is to analyze a conversation and predict the therapy strategy or strategies used. "
        "Before giving your final answer, explain your reasoning step-by-step, showing how earlier parts "
        "of the conversation led to your prediction. The possible strategies are: Question, Restatement or Paraphrasing, "
        "Reflection of feelings, Self-disclosure, Affirmation and Reassurance, Providing Suggestions, Information, and Others. "
        "Finally, output the final predicted strategies as a comma-separated list."
    )
    system_prompt = f"{task_description}\n\nExamples:\n{example_prompts}\nBased on the above, predict the strategies for the following conversation."
    return system_prompt

system_prompt = create_system_prompt_custom()
print("=== System Prompt ===")
print(system_prompt)
print(len(system_prompt))
print("\n"+"=" * 50)

# -----------------------------
# Helper: Convert list of labels to a multi-hot vector.
# -----------------------------
def convert_labels_to_vector(label_list):
    vector = [0] * len(strategy_labels)
    for label in label_list:
        label = label.strip()
        if label in label2id:
            vector[label2id[label]] = 1
    return vector

# -----------------------------
# Generate Prediction with Chain-of-Thought Reasoning using a causal LM
# -----------------------------
def generate_prediction(input_text, system_prompt, tokenizer, model, max_new_tokens=256):
    # Build the full prompt.
    full_prompt = f"{system_prompt}\n\nInput: {input_text}\n\nChain-of-Thought and Final Answer:"
    inputs = tokenizer(full_prompt, return_tensors="pt", truncation=True, max_length=2048)
    inputs = {k: v.to(model.device) for k, v in inputs.items()}
    
    output_ids = model.generate(
        **inputs,
        max_new_tokens=max_new_tokens,
        num_beams=5,
        early_stopping=True
    )
    prediction = tokenizer.decode(output_ids[0], skip_special_tokens=True)
    return prediction.strip()


# -----------------------------
# Evaluate on Test Data and Compute Metrics
# -----------------------------
all_true_vectors = []
all_pred_vectors = []

print("\n=== Testing on Test Data ===\n")
for idx, test_example in enumerate(raw_datasets["test"]):
    
    if idx>=20: break
        
    test_input = test_example["text"]
    expected_label_str = test_example["label"]
    
    try:
        true_labels = ast.literal_eval(expected_label_str)
    except Exception:
        true_labels = [expected_label_str]
    true_labels = [label.strip() for label in true_labels]
    
    generated_output = generate_prediction(test_input, system_prompt, tokenizer, model)
    
    # Extract final strategies from output.
    if "Final Answer:" in generated_output:
        final_part = generated_output.split("Final Answer:")[-1]
        pred_labels = [lab.strip() for lab in final_part.split(",") if lab.strip()]
    else:
        pred_labels = [lab.strip() for lab in generated_output.split(",") if lab.strip()]
    
    true_vector = convert_labels_to_vector(true_labels)
    pred_vector = convert_labels_to_vector(pred_labels)
    
    all_true_vectors.append(true_vector)
    all_pred_vectors.append(pred_vector)
    
    print(f"Example {idx+1}:")
    print("Input:")
    print(test_input)
    print("\nChain-of-Thought and Prediction:")
    print(generated_output)
    print(f"\nExpected Strategies: {true_labels}\n")
    print("-" * 50)

f1 = f1_score(all_true_vectors, all_pred_vectors, average="micro", zero_division=0)
precision = precision_score(all_true_vectors, all_pred_vectors, average="micro", zero_division=0)
recall = recall_score(all_true_vectors, all_pred_vectors, average="micro", zero_division=0)

print("\n=== Overall Metrics ===")
print(f"F1 Score: {f1:.4f}")
print(f"Precision: {precision:.4f}")
print(f"Recall: {recall:.4f}")


tokenizer_config.json:   0%|          | 0.00/1.65k [00:00<?, ?B/s]

tokenizer.model:   0%|          | 0.00/493k [00:00<?, ?B/s]

added_tokens.json:   0%|          | 0.00/51.0 [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/443 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/660 [00:00<?, ?B/s]

model.safetensors.index.json:   0%|          | 0.00/23.9k [00:00<?, ?B/s]

model-00001-of-00003.safetensors:   0%|          | 0.00/4.94G [00:00<?, ?B/s]

KeyboardInterrupt: 

In [44]:
raw_datasets["test"][5]

{'text': "sys(Self-disclosure, Affirmation and Reassurance): It's like we live the same life. I also have no car to escape! It seems so small, but it's such a huge stressor when you feel trapped in an environment you're not positive in. I feel your pain and I empathize with you completely. It'll be hard but I hope you can make it through it throughout the holidays and enjoy yourself some. Are there any pros to going back home? Any pets? usr: Thank you I appreciate that. I will be fine making it over the thanksgiving break but I am more nervous about covid-19 sending us home for good. Not many to be honest. I have a hamster but he is at school with me so nothing at home to go back to sys(Restatement or Paraphrasing, Providing Suggestions): It sounds like Covid- 19 is going to be a personal stressor for you. It's such a strange thing to have to live with already, the pandemic, and i'm sorry that it might end up pushing you where you don't want to be. Could you bring your hamster home wit